## Generate Chips from Sliding Window

In [123]:
import numpy as np
import PIL.Image
import gdal

from skimage.transform import resize, rotate

In [124]:
def chip_is_empty(img):
    """
    Check if region contains a label
    """
    labels_unique = np.unique(img)
    if 0 in labels_unique and len(labels_unique) == 1:
        return True
    else:
        return False


def load_file(path, resizeTo=None):
    """
    Read .tif file
    """
    dataSource = gdal.Open(path)
    if (dataSource is not None):
        bands = []
        for index in range(1, dataSource.RasterCount + 1):
            band = dataSource.GetRasterBand(index).ReadAsArray()
            if (resizeTo):
                band = resize(band, resizeTo, preserve_range=True, anti_aliasing=True).astype(np.int16)
            bands.append(band)
        image = np.dstack(bands)
        return image
    else:
        return None


def save_image(data, path, area_name, x, y):
    """
    Save chip as a .tif
    """
    print(data.shape)
    im = PIL.Image.fromarray((data * 255).astype(np.uint8))
    im.save(f'{path}{area_name}_{x}_{y}.tif')


def sliding_window(image, step=(256, 256), windowSize=(512, 512), windowResize=None):
    """
    Generator for each chip the sliding window
    """
    # image = np.zeros([7275, 9554], dtype=np.int16)
    image_cols = image.shape[1]
    image_rows = image.shape[0]
	## 0 to end by 256
    for y in range(0, image_rows, step[0]):
        for x in range(0, image_cols, step[1]):
            # yield the current window
            window = image[ y:y + windowSize[0], x:x + windowSize[1]]
            ## need ifs for edge cases
            yield (x, y, window, windowSize) # do we just have black pixels for the end??


def gen_data(image_path, output_path, chip_size=256, channels=4, rotate=False, flip=False):
    image_data = load_file(image_path) ## load full image
    image = image_data
    # image = None

    # pad image to avoid the loss of border samples
    pad_size=int(chip_size*0.25)

    count = 0
    for (x, y, window, dimension) in sliding_window(image):
        print(x, y, dimension)
        count += 1
        ## now we know why we need the resize,
        ## if the end of the col/row is smaller than the chip size
        ## then we have to resize it back to 512x512.
        train = np.array(window[:, :, : channels], dtype=np.int16)
        ## labels = np.array(window[:, :, -1:], dtype=np.int8)
        
        # train = normalize(train)
        # labels_unique = np.unique(labels)

        # removed check if empty
        image_raw = train # np.dstack([train, labels]) ## same as image?
        images_daugmentation = [image_raw]

        for i in images_daugmentation: ## list of raw + augmented images
            new_train = i[:,:,:channels]
            new_labels = None #np.array(i[:,:,-1:], dtype=np.int8)

            # removed check if empty
            # np.clip(new_labels, 0, None, out=new_labels) ## negative values to 0
            # new_data = (new_train, new_labels)
            save_image(new_train, output_path, 'SA', x, y)
        # removed check if empty
    print('Count: ', count)
    return

In [132]:
output_path = '/content/data/'# make data folder first, will take a while to show up
image_path = '/content/image1t.tif'
gen_data(image_path, output_path)
# sliding_window(None)    

0 0 (512, 512)
(512, 512, 4)
256 0 (512, 512)
(512, 512, 4)
512 0 (512, 512)
(512, 512, 4)
768 0 (512, 512)
(512, 512, 4)
1024 0 (512, 512)
(512, 512, 4)
1280 0 (512, 512)
(512, 512, 4)
1536 0 (512, 512)
(512, 512, 4)
1792 0 (512, 512)
(512, 512, 4)
2048 0 (512, 512)
(512, 512, 4)
2304 0 (512, 512)
(512, 512, 4)
2560 0 (512, 512)
(512, 512, 4)
2816 0 (512, 512)
(512, 512, 4)
3072 0 (512, 512)
(512, 512, 4)
3328 0 (512, 512)
(512, 512, 4)
3584 0 (512, 512)
(512, 512, 4)
3840 0 (512, 512)
(512, 512, 4)
4096 0 (512, 512)
(512, 512, 4)
4352 0 (512, 512)
(512, 512, 4)
4608 0 (512, 512)
(512, 512, 4)
4864 0 (512, 512)
(512, 512, 4)
5120 0 (512, 512)
(512, 512, 4)
5376 0 (512, 512)
(512, 512, 4)
5632 0 (512, 512)
(512, 512, 4)
5888 0 (512, 512)
(512, 512, 4)
6144 0 (512, 512)
(512, 512, 4)
6400 0 (512, 512)
(512, 512, 4)
6656 0 (512, 512)
(512, 512, 4)
6912 0 (512, 512)
(512, 512, 4)
7168 0 (512, 512)
(512, 512, 4)
7424 0 (512, 512)
(512, 512, 4)
7680 0 (512, 512)
(512, 512, 4)
7936 0 (512, 5

## Label Studio JSON_MIN to TIFF

In [31]:
import json, os, glob

import gdal
from PIL import Image, ImageDraw
import numpy as np
import skimage

In [3]:
json_path = '/content/project-2-at-2022-04-07-20-01-8f5e5b1a.json'
# a JSON_MIN exported label
# the regular JSON contains both the image and the label

In [15]:
with open(json_path) as json_file:
    data = json.load(json_file) # list of labeled images
    print(data[0].keys()) # keys for each label

dict_keys(['image', 'id', 'ellipse', 'labels', 'polygon', 'brush', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time'])


In [17]:
data[0]['image'] # URL

'/data/upload/2/8d83f97f-img4096_5120.jpeg'

In [27]:
data[0]['ellipse'] # Main labeling tool, list of ellipse annotations

[{'original_height': 1024,
  'original_width': 1024,
  'radiusX': 14.466666666666628,
  'radiusY': 14.133333333333297,
  'rotation': 0,
  'x': 61.53333333333328,
  'y': 66.6666666666666},
 {'original_height': 1024,
  'original_width': 1024,
  'radiusX': 16.00000000000001,
  'radiusY': 15.733333333333396,
  'rotation': 0,
  'x': 29.73333333333332,
  'y': 57.46666666666662},
 {'original_height': 1024,
  'original_width': 1024,
  'radiusX': 14.466666666666733,
  'radiusY': 14.399999999999993,
  'rotation': 0,
  'x': 68.99999999999994,
  'y': 34.00000000000001},
 {'original_height': 1024,
  'original_width': 1024,
  'radiusX': 15.733333333333348,
  'radiusY': 15.933333333333321,
  'rotation': 0,
  'x': 38.13333333333332,
  'y': 26.066666666666666}]

In [99]:
ellipse1 = data[0]['ellipse'][0]
ellipse1

{'original_height': 1024,
 'original_width': 1024,
 'radiusX': 14.466666666666628,
 'radiusY': 14.133333333333297,
 'rotation': 0,
 'x': 61.53333333333328,
 'y': 66.6666666666666}

In [126]:
height = ellipse1['original_height']
width = ellipse1['original_width']
# Convert from 0-100 to original width
# https://labelstud.io/tags/ellipselabels.html
r_radius = ellipse1['radiusX'] * width / 100
c_radius = ellipse1['radiusY'] * height / 100
r = ellipse1['x'] * width / 100
c = ellipse1['y'] * width / 100
shape = [height, width]

In [127]:
# https://github.com/matterport/Mask_RCNN/issues/2154, 
mask = np.zeros([height, width, 1], dtype=np.uint8)
rr, cc = skimage.draw.ellipse(r, c, r_radius, c_radius, shape=shape, rotation=0.0)
# for i in range(len(cc)):
    # print(rr[i], cc[i])
    # These are the (x, y) coord pairs
mask[rr, cc, 0] = 1
mask.astype(bool)
print(np.unique(mask, return_counts=True))
mask.shape

(array([0, 1], dtype=uint8), array([981222,  67354]))


(1024, 1024, 1)

In [130]:
# Save Image Mask
# https://stackoverflow.com/questions/60138697/typeerror-cannot-handle-this-data-t
# https://stackoverflow.com/questions/55319949/pil-typeerror-cannot-handle-this-data-type
np_img = np.squeeze(mask, axis=2)  # axis=2 is channel dimension 
im = PIL.Image.fromarray((np_img * 255).astype(np.uint8))
im.save("your_file.tiff") # export as tiff, jpg, png

In [21]:
data[0]['polygon']; # For partials

In [22]:
data[0]['brush']; # For partials